In [2]:
import requests
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium import webdriver
import pickle
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

#############################################################################################3
driver = webdriver.Chrome()

COOKIE_FILE = "tradingview_cookies.pkl"


def save_cookies():
    """
        Save Cookie
    """
    with open(COOKIE_FILE, "wb") as file:
        pickle.dump(driver.get_cookies(), file)
    # log_message("Save cookies")


def load_cookies():
    """
        Load Cookies
    """
    try:
        with open(COOKIE_FILE, "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)
        # log_message("Set cookies")
    except FileNotFoundError:
        # log_message("Cookies Not found")
        print('Cookie Not Found')


try:

    driver.get("https://judotv.com/")

    load_cookies()

    driver.refresh()
    input('****')
    try:
        login_button = driver.find_element(By.XPATH,
                                           '//*[@id="judo-tv"]/div/div[2]/div/header/div[2]/div[1]/div[2]/button')
        login_button.click()
        sleep(5)
        input('---')
        username = driver.find_element(By.NAME, "Username")
        password = driver.find_element(By.NAME, "Password")
        username.send_keys("yivic44760@cotigz.com")  # username
        password.send_keys("mobin2003")  # password
        password.send_keys(Keys.RETURN)

        sleep(5)
        driver.switch_to.default_content()
        # log_message("Login")
        sleep(10)
        save_cookies()
    except:
        print('Login')
except:
    print('cant Login')

input('ok ?')
try:

    allow_button = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
    )
    allow_button.click()
    print("1 Cookie Button")
except:
    print("0 Cookie Button")
#####################################################################################################



**** 
--- 


Login


ok ? 


1 Cookie Button


In [50]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def analyze_judo_match(driver):
    """
    تحلیل صفحه مسابقه جودو با استفاده از Selenium WebDriver و Full XPath برای استخراج جودوکاران، امتیازات، رویدادها و برنده.
    فرض می‌شود صفحه کامل بارگذاری شده، اما ممکن است نیاز به کلیک روی تب Events باشد.
    
    Args:
        driver: شیء WebDriver از Selenium که صفحه مسابقه را بارگذاری کرده است
        
    Returns:
        dict: دیکشنری شامل اطلاعات مسابقه (جودوکاران، امتیازات، رویدادها با زمان‌بندی، برنده) یا خطا
    """
    try:
        # انتظار برای بارگذاری کامل صفحه
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        print("Page loaded successfully")

        # بررسی وجود صفحه لاگین
        # تلاش برای کلیک روی تب Events (در صورت وجود)
        try:
            events_tab = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Events") or contains(@class, "event-tab")]'))
            )
            events_tab.click()
            time.sleep(2)  # انتظار برای بارگذاری تب
            print("Clicked on Events tab")
        except (TimeoutException, NoSuchElementException):
            print("No Events tab found or not clickable")

        # استخراج نام و کشور جودوکاران
        judoka_elements = driver.find_elements(By.XPATH, '/html/body//div[contains(@class, "font-medium") or contains(@class, "font-bold") and contains(@class, "text-") and not(contains(@class, "text-neutral-100"))]')
        judokas = []
        valid_names = ['Eric TAKABATAKE', 'Walide KHYAR']
        seen_names = set()
        for elem in judoka_elements:
            name = elem.text.strip().replace('\n', ' ').replace('\u2009', ' ')
            if not name or name not in valid_names or name in seen_names:
                continue
            seen_names.add(name)
            try:
                country = 'Unknown'
                for xpath in [
                    './following::div[contains(@class, "text-xs") and contains(@class, "font-bold") and contains(@class, "uppercase")][1]',
                    './preceding::div[contains(@class, "text-xs") and contains(@class, "font-bold") and contains(@class, "uppercase")][1]',
                    '/html/body//div[contains(text(), "' + name + '")]/following::div[contains(@class, "text-xs") and contains(@class, "font-bold") and string-length(text()) <= 3][1]',
                    './parent::div//div[contains(@class, "text-neutral-100") and string-length(text()) <= 3]'
                ]:
                    try:
                        country_elem = elem.find_element(By.XPATH, xpath)
                        country_text = country_elem.text.strip().upper()
                        if country_text and len(country_text) <= 3:
                            country = country_text
                            break
                    except NoSuchElementException:
                        continue
                if country == 'Unknown':
                    print(f"Country not found for {name}")
            except NoSuchElementException:
                print(f"Country XPath failed for {name}")
            judokas.append({'name': name, 'country': country})
            print(f"Found judoka: {name} ({country})")

        if len(judokas) != 2:
            return {'error': f'Expected exactly 2 judokas, found: {judokas}'}

        # استخراج امتیازات
        score_elements = driver.find_elements(By.XPATH, '/html/body//div[contains(@class, "grid") and contains(@class, "text-center") and (contains(@class, "grid-cols-3") or contains(@class, "grid-cols-4"))]')
        scores = []
        for i, score_elem in enumerate(score_elements):
            if i >= len(judokas):
                print(f"Warning: More score elements than judokas")
                break
            score_data = {'name': judokas[i]['name'], 'country': judokas[i]['country'], 'ippon': 0, 'waza_ari': 0, 'yuko': 0, 'shido': 0}
            cells = score_elem.find_elements(By.XPATH, './/div[contains(@class, "text-") or contains(@class, "font-bold")]')
            for cell in cells:
                cell_text = cell.text.strip()
                cell_classes = cell.get_attribute('class').split()
                try:
                    title = cell.get_attribute('title') or ''
                    if 'Ippon' in title and cell_text.isdigit():
                        score_data['ippon'] = int(cell_text)
                    elif 'Waza-ari' in title and cell_text.isdigit():
                        score_data['waza_ari'] = int(cell_text)
                    elif 'Yuko' in title and cell_text.isdigit():
                        score_data['yuko'] = int(cell_text)
                    elif 'Penalty' in title and 'bg-neutral-100/0' not in cell_classes:
                        score_data['shido'] += 1
                except:
                    continue
            scores.append(score_data)
            print(f"Scores for {score_data['name']}: {score_data}")

        if len(scores) != 2:
            return {'error': f'Expected exactly 2 score sets, found: {scores}'}

        # استخراج رویدادها
        events = []
        # تست چندین XPath برای رویدادها، با فیلتر کردن Video.js
        event_xpaths = [
            '/html/body//div[contains(@class, "grid") and contains(@class, "gap-") and contains(@class, "grid-cols-[1fr,auto,1fr]") and not(contains(@class, "vjs-"))]',
            '/html/body//div[contains(@class, "event") and not(contains(@class, "vjs-"))]',
            '/html/body//ul[contains(@class, "event") or contains(@class, "log") and not(contains(@class, "vjs-"))]',
            '/html/body//div[contains(@class, "timeline") and not(contains(@class, "vjs-"))]',
            '/html/body//div[contains(text(), "Ippon") or contains(text(), "Shido") or contains(text(), "Waza-ari")]/parent::div'
        ]
        for xpath in event_xpaths:
            event_elements = driver.find_elements(By.XPATH, xpath)
            if event_elements:
                print(f"Found {len(event_elements)} event elements with XPath: {xpath}")
                break
        else:
            event_elements = []
            print("No event elements found with any XPath")

        for event_elem in event_elements:
            try:
                # جستجوی زمان و تکنیک
                time_elem = event_elem.find_element(By.XPATH, './/div[contains(@class, "text-sm") and contains(@class, "font-bold")] | .//span[contains(@class, "time") or text()[contains(., ":")]]')
                technique_elem = event_elem.find_element(By.XPATH, './/h4[contains(@class, "font-bold")] | .//span[contains(@class, "technique") or contains(text(), "Ippon") or contains(text(), "Shido") or contains(text(), "Waza-ari")]')
                time = time_elem.text.strip()
                technique = technique_elem.text.strip()
                # شناسایی جودوکار
                judoka_side = event_elem.find_elements(By.XPATH, './/div[contains(@class, "bg-white") or contains(@class, "bg-blue") or contains(text(), "TAKABATAKE") or contains(text(), "KHYAR")]')
                if judoka_side:
                    judoka_name = scores[0]['name'] if 'TAKABATAKE' in judoka_side[0].text or 'bg-white' in judoka_side[0].get_attribute('class') else scores[1]['name']
                    event_type = 'Technique' if technique.lower() in ['ippon', 'waza-ari', 'yuko'] else 'Penalty' if technique.lower() == 'shido' else 'Other'
                    events.append({
                        'time': time,
                        'type': event_type,
                        'technique': technique,
                        'judoka': judoka_name
                    })
                    print(f"Event found: {technique} by {judoka_name} at {time}")
                else:
                    print(f"No judoka side identified for event: {technique}")
            except NoSuchElementException:
                print(f"Failed to parse event element: {event_elem.get_attribute('outerHTML')[:100]}...")
                continue

        # استنباط رویدادها از امتیازات
        if not events:
            print("No events found in DOM. Inferring events from scores.")
            for score in scores:
                for _ in range(score['ippon']):
                    events.append({
                        'time': 'Unknown',
                        'type': 'Technique',
                        'technique': 'Ippon',
                        'judoka': score['name']
                    })
                for _ in range(score['waza_ari']):
                    events.append({
                        'time': 'Unknown',
                        'type': 'Technique',
                        'technique': 'Waza-ari',
                        'judoka': score['name']
                    })
                for _ in range(score['yuko']):
                    events.append({
                        'time': 'Unknown',
                        'type': 'Technique',
                        'technique': 'Yuko',
                        'judoka': score['name']
                    })
                for _ in range(score['shido']):
                    events.append({
                        'time': 'Unknown',
                        'type': 'Penalty',
                        'technique': 'Shido',
                        'judoka': score['name']
                    })

        # تعیین برنده
        winner = None
        if scores[0]['ippon'] > scores[1]['ippon']:
            winner = f"{scores[0]['name']} ({scores[0]['country']})"
        elif scores[1]['ippon'] > scores[0]['ippon']:
            winner = f"{scores[1]['name']} ({scores[1]['country']})"
        elif scores[0]['waza_ari'] > scores[1]['waza_ari']:
            winner = f"{scores[0]['name']} ({scores[0]['country']})"
        elif scores[1]['waza_ari'] > scores[0]['waza_ari']:
            winner = f"{scores[1]['name']} ({scores[1]['country']})"
        elif scores[0]['yuko'] > scores[1]['yuko']:
            winner = f"{scores[0]['name']} ({scores[0]['country']})"
        elif scores[1]['yuko'] > scores[0]['yuko']:
            winner = f"{scores[1]['name']} ({scores[1]['country']})"
        elif scores[0]['shido'] < scores[1]['shido']:
            winner = f"{scores[0]['name']} ({scores[0]['country']})"
        elif scores[1]['shido'] < scores[0]['shido']:
            winner = f"{scores[1]['name']} ({scores[1]['country']})"
        else:
            winner = "Draw or insufficient data"

        # ایجاد خروجی
        result = {
            'judokas': [
                {
                    'name': scores[0]['name'],
                    'country': scores[0]['country'],
                    'scores': {
                        'ippon': scores[0]['ippon'],
                        'waza_ari': scores[0]['waza_ari'],
                        'yuko': scores[0]['yuko'],
                        'shido': scores[0]['shido']
                    }
                },
                {
                    'name': scores[1]['name'],
                    'country': scores[1]['country'],
                    'scores': {
                        'ippon': scores[1]['ippon'],
                        'waza_ari': scores[1]['waza_ari'],
                        'yuko': scores[1]['yuko'],
                        'shido': scores[1]['shido']
                    }
                }
            ],
            'events': events,
            'winner': winner
        }

        return result

    except Exception as e:
        return {'error': f'Analysis failed: {str(e)}'}

In [51]:
with open('watch.txt', 'r') as file:
    lines = file.readlines()
for line in lines[1:2] :
    driver.get(line)
    # element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/main/div[2]/div/div[2]/div[2]')
    # # گرفتن سورس HTML آن عنصر
    # html_source = element.get_attribute('outerHTML')
    print(analyze_judo_match(driver))

Page loaded successfully
No Events tab found or not clickable
Found judoka: Eric TAKABATAKE (BRA)
Found judoka: Walide KHYAR (FRA)
Scores for Eric TAKABATAKE: {'name': 'Eric TAKABATAKE', 'country': 'BRA', 'ippon': 0, 'waza_ari': 0, 'yuko': 0, 'shido': 1}
Scores for Walide KHYAR: {'name': 'Walide KHYAR', 'country': 'FRA', 'ippon': 1, 'waza_ari': 0, 'yuko': 0, 'shido': 1}
No event elements found with any XPath
No events found in DOM. Inferring events from scores.
{'judokas': [{'name': 'Eric TAKABATAKE', 'country': 'BRA', 'scores': {'ippon': 0, 'waza_ari': 0, 'yuko': 0, 'shido': 1}}, {'name': 'Walide KHYAR', 'country': 'FRA', 'scores': {'ippon': 1, 'waza_ari': 0, 'yuko': 0, 'shido': 1}}], 'events': [{'time': 'Unknown', 'type': 'Penalty', 'technique': 'Shido', 'judoka': 'Eric TAKABATAKE'}, {'time': 'Unknown', 'type': 'Technique', 'technique': 'Ippon', 'judoka': 'Walide KHYAR'}, {'time': 'Unknown', 'type': 'Penalty', 'technique': 'Shido', 'judoka': 'Walide KHYAR'}], 'winner': 'Walide KHYAR 